Code from https://github.com/vrdi/shortbursts-gingles/blob/main/state_experiments/sb_runs.py

In [34]:
from gingleator import Gingleator

import pandas as pd
import geopandas as gpd
import maup
import time
import numpy as np
import pickle

import matplotlib.pyplot as plt
from gerrychain import Graph, Partition, proposals, updaters, constraints, accept, MarkovChain, Election
from gerrychain.updaters import cut_edges, Tally
from gerrychain.proposals import recom
from functools import partial

In [21]:
score_functs = {0: None, 1: Gingleator.reward_partial_dist, 
                2: Gingleator.reward_next_highest_close,
                3: Gingleator.penalize_maximum_over,
                4: Gingleator.penalize_avg_over}

In [22]:
BURST_LEN = 10
NUM_DISTRICTS = 8
ITERS = 100
POP_COL = "TOTPOP20"
N_SAMPS = 10
SCORE_FUNCT = None
EPS = 0.045
MIN_POP_COL = 'HISP'

In [23]:
graph = Graph.from_file('./data/cleaned/final data/CO.shp')

In [24]:
graph.nodes[0]

{'boundary_node': False,
 'area': 39373310.67444259,
 'STATEFP': '08',
 'COUNTYFP': '001',
 'VTDST': '001226',
 'NAME': 'Adams 226',
 'PRECINCT': '6253001226',
 'PRES20D': 60,
 'PRES20R': 128,
 'SEN20D': 60,
 'SEN20R': 131,
 'TOTPOP20': 368,
 'VAP20': 285,
 'PRES16D': 59.0,
 'PRES16R': 110.0,
 'SEN16D': 72.0,
 'SEN16R': 97.0,
 'GOV18D': 52.0,
 'GOV18R': 80.0,
 'ATG18D': 50.0,
 'ATG18R': 83.0,
 'SOS18D': 52.0,
 'SOS18R': 80.0,
 'TRE18D': 51.0,
 'TRE18R': 76.0,
 'HISP': 141,
 'NH_WHITE': 204,
 'NH_BLACK': 0,
 'NH_AMIN': 1,
 'NH_ASIAN': 12,
 'NH_NHPI': 0,
 'NH_OTHER': 0,
 'NH_2MORE': 10,
 'CD': 8,
 'geometry': <POLYGON ((527454.509 4424739.321, 527457.779 4424663.821, 527457.48 4424594...>}

In [25]:
my_updaters = {"population" : Tally(POP_COL, alias="population"),
               "VAP": Tally("VAP20", alias="VAP"),
               "HISP": Tally("HISP", alias="HISP"),
               "cut_edges": cut_edges}

In [26]:
total_pop = sum([graph.nodes()[n][POP_COL] for n in graph.nodes()])

In [27]:
init_partition = Partition(graph, assignment="CD", updaters=my_updaters)

gingles = Gingleator(init_partition, pop_col=POP_COL,
                     threshold=0.5, score_funct=SCORE_FUNCT, epsilon=EPS,
                     minority_perc_col="{}_perc".format(MIN_POP_COL))

gingles.init_minority_perc_col(MIN_POP_COL, "VAP", 
                               "{}_perc".format(MIN_POP_COL))

HISP_perc


In [35]:
num_bursts = int(ITERS/BURST_LEN)

print("Starting Short Bursts Runs", flush=True)

for n in range(N_SAMPS):
    sb_obs = gingles.short_burst_run(num_bursts=num_bursts, num_steps=BURST_LEN,
                                     maximize=True, verbose=False)
    print("\tFinished chain {}".format(n), flush=True)

    print("\tSaving results", flush=True)

    f_out = "ShortBurstOutput.npy"
    
    np.save(f_out, sb_obs[1])

    f_out_part = "ShortBurstPartition.p"

    max_stats = {"VAP": sb_obs[0][0]["VAP"],
                 "HISP": sb_obs[0][0]["HISP"],}

    with open(f_out_part, "wb") as f_out:
        pickle.dump(max_stats, f_out)

Starting Short Bursts Runs
	Finished chain 0
	Saving results
	Finished chain 1
	Saving results
	Finished chain 2
	Saving results
	Finished chain 3
	Saving results
	Finished chain 4
	Saving results
	Finished chain 5
	Saving results
	Finished chain 6
	Saving results
	Finished chain 7
	Saving results
	Finished chain 8
	Saving results
	Finished chain 9
	Saving results
